In [1]:
import pandas as pd
import numpy as np

In [6]:
pivot_data4 = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_data4.csv')
activity_type_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/report_full_activity_type.csv')

In [3]:
import pandas as pd
import numpy as np

def classify_motion_row(row):
    # Extract relevant data from the row
    accel_x = row['accel_x']
    accel_y = row['accel_y']
    accel_z = row['accel_z']
    gyro_x = row['gyro_x']
    gyro_y = row['gyro_y']
    gyro_z = row['gyro_z']
    orientation_x = row['orientation_x']
    orientation_y = row['orientation_y']
    orientation_z = row['orientation_z']
    
    # Classify motion based on accelerometer and gyroscope readings
    if np.any((accel_x != 0) | (accel_y != 0) | (accel_z != 0)):
        # Check if all gyroscope readings are zero
        if not np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
            # Check orientation for refining moving motion
            if orientation_z > 90 or orientation_z < -90:
                motion_type = "Vertical Movement"
            elif orientation_x > 90 or orientation_x < -90:
                motion_type = "Horizontal Movement"
            else:
                motion_type = "Moving"
        else:
            # If gyroscope readings are not all zero, classify as "Turning/Tilting"
            motion_type = "Turning/Tilting"
    elif np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
        # If accelerometer readings are all zero, check gyroscope readings
        motion_type = "Turning/Tilting"
    else:
        # If both accelerometer and gyroscope readings are zero, classify as "Stationary"
        motion_type = "Stationary"

    return pd.Series([motion_type])

# Apply the function row-wise to the dataframe
pivot_data4['motion_type'] = pivot_data4.apply(classify_motion_row, axis=1)


In [16]:
from sklearn.preprocessing import LabelEncoder

# Create and fit the LabelEncoder
label_encoder_motion = LabelEncoder()
pivot_data4['motion_encoded'] = label_encoder_motion.fit_transform(pivot_data4['motion_type'])

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM


# Split the data into features (X) and target variable (y)
X = pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]
y = pivot_data4['motion_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define the model
model = Sequential([
    Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.001), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.001)),  # Removed extra parenthesis here
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(np.unique(pivot_data4['motion_encoded'])), activation='softmax') 
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Encode class labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Train the model with class weights and early stopping
history = model.fit(X_train_scaled, y_train_encoded, epochs=50, validation_data=(X_val_scaled, y_val), class_weight=class_weight_dict)

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/motion.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

Epoch 1/50
3474/3474 [==============================] - 17s 4ms/step - loss: 1.7877 - accuracy: 0.7094 - val_loss: 0.6928 - val_accuracy: 0.9360
Epoch 2/50
3474/3474 [==============================] - 15s 4ms/step - loss: 1.2126 - accuracy: 0.8523 - val_loss: 0.6563 - val_accuracy: 0.9335
Epoch 3/50
3474/3474 [==============================] - 15s 4ms/step - loss: 1.0103 - accuracy: 0.8893 - val_loss: 0.5470 - val_accuracy: 0.9315
Epoch 4/50
3474/3474 [==============================] - 14s 4ms/step - loss: 0.9368 - accuracy: 0.9050 - val_loss: 0.5258 - val_accuracy: 0.9341
Epoch 5/50
3474/3474 [==============================] - 13s 4ms/step - loss: 0.9211 - accuracy: 0.9129 - val_loss: 0.4388 - val_accuracy: 0.9401
Epoch 6/50
3474/3474 [==============================] - 15s 4ms/step - loss: 0.8740 - accuracy: 0.9136 - val_loss: 0.4991 - val_accuracy: 0.9363
Epoch 7/50
3474/3474 [==============================] - 15s 4ms/step - loss: 0.8844 - accuracy: 0.9166 - val_loss: 0.4533 - val_ac

In [11]:
def classify_activity(row):
    motion_type = row['motion_type']
    if motion_type == "Vertical Movement":
        predicted_activity = ["Running", "Walking", "Walking & party", "Football"]
    elif motion_type == "Horizontal Movement":
        predicted_activity = ["In vehicle", "Train", "In Bus", "On Bus stop"]
    elif motion_type == "Moving":
        predicted_activity = ["Cooking", "Meeting", "Movie", "Shopping & wearing"]
    elif motion_type == "Turning/Tilting":
        predicted_activity = ["Phone was out of the pocket (forgot)", "Took off glasses"]
    elif motion_type == "Stationary":
        predicted_activity = ["At home", "Pause", "Shop", "Work", "Eating", "Sleeping", "Watching TV", "Eating", "Picnic", "Video Games"]
    return predicted_activity

# Assuming 'pivot_data4' is your DataFrame
pivot_data4['predicted_activity'] = pivot_data4.apply(classify_activity, axis=1)

In [12]:
pivot_data4

,timestamp,step_counter,step_detector,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,linear_accel_x,...,orientation_x,orientation_y,orientation_z,rotation_scalar,rotation_i,rotation_j,rotation_k,rotation_angle,motion_type,predicted_activity
0,2017-06-29 07:45:56.262,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
1,2017-06-29 07:46:01.263,0.0,0.0,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Moving,"[Cooking, Meeting, Movie, Shopping & wearing]"
2,2017-06-29 07:46:01.324,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
3,2017-06-29 07:46:01.329,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.018069,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
4,2017-06-29 07:46:01.402,0.0,0.0,0.000000,0.000000,0.000000,0.004166,-0.013275,0.006165,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Turning/Tilting,"[Phone was out of the pocket (forgot), Took of..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173657,2017-07-13 20:21:35.009,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
173658,2017-07-13 20:21:35.020,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
173659,2017-07-13 20:21:35.060,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."
173660,2017-07-13 20:21:35.124,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,"[At home, Pause, Shop, Work, Eating, Sleeping,..."


In [7]:
activity_types = activity_type_full['activity_type'].unique()

In [8]:
activity_types

array(['Video games', 'In computer', 'At home', 'Eat', 'Sleep',
       'Watching TV', 'In vehicle', 'Picnic ', 'Walk', 'Running',
       'Phone was out of the pocket (forgot)', 'Train', 'Cooking',
       'In bus', 'Meeting', 'Pause', 'Movie', 'Football', 'Shop',
       'Shopping& wearing', 'Walking&party', 'On bus stop',
       'Took off glasses', 'Work'], dtype=object)

In [4]:
pivot_data4_pathway = '/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_data4.csv'

pivot_data4.to_csv(pivot_data4_pathway, index=False)

In [5]:
pivot_data4['motion_type'].value_counts()


motion_type
Stationary             156088
Turning/Tilting          8986
Moving                   7795
Vertical Movement         502
Horizontal Movement       291
Name: count, dtype: int64

This diversity of motion_types shows that the function is working. 

In [15]:
from potassium import Potassium, Request, Response
from transformers import pipeline
import torch
import time

app = Potassium("my_app")

# @app.init runs at startup, and initializes the app's context
@app.init
def init():
    # Check if there is a GPU
    device = 0 if torch.cuda.is_available() else -1

    # Defind the model, using the "pipeline" interface from transformers
    bertModel = pipeline('fill-mask', model='bert-base-uncased', device=device)
   
    
    context = {
        "model_b": bertModel,
    }

    return context



# @app.handler is an http post handler running for every call
@app.handler("/bert")
def handler(context: dict, request: Request) -> Response:
    
    # our model is from 'model_b' as defined in the context earlier
    model = context.get("model_b")

    # our prompt is from the 'prompt' key in the user request
    prompt = request.json.get("prompt")

    # the outputs is created by passing the user request to the model
    outputs = model(prompt)

    # returned
    return Response(
        json = {"outputs": outputs}, 
        status=200
    )

if __name__ == "__main__":
    app.serve()

------
Starting Potassium Server 🍌
Running init()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Started 1 workers
Serving at http://0.0.0.0:8000
------
127.0.0.1 - - [22/Mar/2024 15:36:47] "GET / HTTP/1.1" 405 -
